In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 52.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6731fa30e27ac70f434073ac3898ae23f339982e7fd1cee60a5211c8e5318f45
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.2.1-bin-hadoop2.7.tgz


In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

import findspark

findspark.init()

import pyspark

In [7]:
### Initialize the spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

In [8]:
import pyspark.ml as ml

In [10]:
salaries_df = spark.read.csv("salaries.csv",inferSchema=True,header=True)

In [15]:
salaries_df.show(10)

+------+------+
|emp_no|salary|
+------+------+
| 10001| 60117|
| 10002| 65828|
| 10003| 40006|
| 10004| 40054|
| 10005| 78228|
| 10006| 40000|
| 10007| 56724|
| 10008| 46671|
| 10009| 60929|
| 10010| 72488|
+------+------+
only showing top 10 rows



In [20]:
employees_df = spark.read.csv("employees.csv",inferSchema=True,header=True)

In [21]:
employees_df.show(10)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex| hire_date|no_of_projects|Last_performance_rating|left|last_date|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+
|473302|       s0001| 7/25/1953|  Hideyuki| Zallocco|  M| 4/28/1990|             2|                      A|   0|     null|
|475053|       e0002|11/18/1954|     Byong|Delgrande|  F|  9/7/1991|             1|                      C|   0|     null|
| 57444|       e0002| 1/30/1958|     Berry|     Babb|  F| 3/21/1992|             9|                      A|   0|     null|
|421786|       s0001| 9/28/1957|     Xiong| Verhoeff|  M|11/26/1987|             2|                      C|   0|     null|
|282238|       e0003|10/28/1952|Abdelkader|  Baumann|  F| 1/18/1991|             6|                      B|   0|     null|
|263976|       e

In [11]:
titles_df = spark.read.csv("titles.csv",inferSchema=True,header=True)

In [16]:
titles_df.show(10)

+--------+------------------+
|title_id|             title|
+--------+------------------+
|   s0001|             Staff|
|   s0002|      Senior Staff|
|   e0001|Assistant Engineer|
|   e0002|          Engineer|
|   e0003|   Senior Engineer|
|   e0004|  Technique Leader|
|   m0001|           Manager|
+--------+------------------+



In [12]:
departments_df = spark.read.csv("departments.csv",inferSchema=True,header=True)

In [17]:
departments_df.show(10)

+-------+------------------+
|dept_no|         dept_name|
+-------+------------------+
|   d001|         Marketing|
|   d002|           Finance|
|   d003|   Human Resources|
|   d004|        Production|
|   d005|       development|
|   d006|Quality Management|
|   d007|             Sales|
|   d008|          Research|
|   d009|  Customer Service|
+-------+------------------+



In [13]:
dept_emp_df = spark.read.csv("dept_emp.csv",inferSchema=True,header=True)

In [18]:
dept_emp_df.show(10)

+------+-------+
|emp_no|dept_no|
+------+-------+
| 10001|   d005|
| 10002|   d007|
| 10003|   d004|
| 10004|   d004|
| 10005|   d003|
| 10006|   d005|
| 10007|   d008|
| 10008|   d005|
| 10009|   d006|
| 10010|   d004|
+------+-------+
only showing top 10 rows



In [14]:
dept_manager_df = spark.read.csv("dept_manager.csv",inferSchema=True,header=True)

In [19]:
dept_manager_df.show(10)

+-------+------+
|dept_no|emp_no|
+-------+------+
|   d001|110022|
|   d001|110039|
|   d002|110085|
|   d002|110114|
|   d003|110183|
|   d003|110228|
|   d004|110303|
|   d004|110344|
|   d004|110386|
|   d004|110420|
+-------+------+
only showing top 10 rows



In [22]:
#performing joins
df_joins = employees_df.join(dept_emp_df, on ='emp_no',how='left')\
          .join(departments_df,departments_df.dept_no==dept_emp_df.dept_no,how='left').drop(dept_emp_df.dept_no)\
          .join(dept_manager_df,on='emp_no',how='left').drop(dept_manager_df.dept_no)\
          .join(salaries_df,on='emp_no',how='left')\
          .join(titles_df,employees_df.emp_title_id==titles_df.title_id,how='left')
            


In [23]:
df_joins.show(10)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+---------------+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex| hire_date|no_of_projects|Last_performance_rating|left|last_date|dept_no|         dept_name|salary|title_id|          title|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+---------------+
|473302|       s0001| 7/25/1953|  Hideyuki| Zallocco|  M| 4/28/1990|             2|                      A|   0|     null|   d002|           Finance| 40000|   s0001|          Staff|
|475053|       e0002|11/18/1954|     Byong|Delgrande|  F|  9/7/1991|             1|                      C|   0|     null|   d004|        Production| 53422|   e0002|       Engineer|
| 57444|       e0002| 1/30/1958|     Berry|     Babb|  F| 3/21/1992|             9|       

In [ ]:
df_joins.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- emp_title_id: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- no_of_projects: integer (nullable = true)
 |-- Last_performance_rating: string (nullable = true)
 |-- left: integer (nullable = true)
 |-- last_date: string (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- title_id: string (nullable = true)
 |-- title: string (nullable = true)



In [ ]:
df_joins.describe().show()

+-------+------------------+------------+----------+----------+---------+------+---------+------------------+-----------------------+-------------------+---------+-------+----------------+------------------+--------+------------------+
|summary|            emp_no|emp_title_id|birth_date|first_name|last_name|   sex|hire_date|    no_of_projects|Last_performance_rating|               left|last_date|dept_no|       dept_name|            salary|title_id|             title|
+-------+------------------+------------+----------+----------+---------+------+---------+------------------+-----------------------+-------------------+---------+-------+----------------+------------------+--------+------------------+
|  count|            331603|      331603|    331603|    331603|   331603|331603|   331603|            331603|                 331603|             331603|    33021| 331603|          331603|            331603|  331603|            331603|
|   mean|253332.60502468314|        null|      null|    

In [ ]:
df_joins.count()

331603

In [ ]:
df_joins.cache()

DataFrame[emp_no: int, emp_title_id: string, birth_date: string, first_name: string, last_name: string, sex: string, hire_date: string, no_of_projects: int, Last_performance_rating: string, left: int, last_date: string, dept_no: string, dept_name: string, salary: int, title_id: string, title: string]

In [24]:
df = df_joins

In [ ]:
df.show(10)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+---------------+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex| hire_date|no_of_projects|Last_performance_rating|left|last_date|dept_no|         dept_name|salary|title_id|          title|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+---------------+
|473302|       s0001| 7/25/1953|  Hideyuki| Zallocco|  M| 4/28/1990|             2|                      A|   0|     null|   d002|           Finance| 40000|   s0001|          Staff|
|475053|       e0002|11/18/1954|     Byong|Delgrande|  F|  9/7/1991|             1|                      C|   0|     null|   d004|        Production| 53422|   e0002|       Engineer|
| 57444|       e0002| 1/30/1958|     Berry|     Babb|  F| 3/21/1992|             9|       

In [25]:
df.columns

['emp_no',
 'emp_title_id',
 'birth_date',
 'first_name',
 'last_name',
 'sex',
 'hire_date',
 'no_of_projects',
 'Last_performance_rating',
 'left',
 'last_date',
 'dept_no',
 'dept_name',
 'salary',
 'title_id',
 'title']

In [26]:
continuous_features = ['no_of_projects','salary']

categorical_features = ['last_performance_ratings', 'dept_name', 'left_',
                      'title']


In [27]:
#Encoding all categorical features
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, PolynomialExpansion, VectorIndexer

In [28]:
# create object of StringIndexer class and specify input and output column
SI_Last_performance_rating = StringIndexer(inputCol='Last_performance_rating',outputCol='Last_performance_rating_Index')
SI_dept_name = StringIndexer(inputCol='dept_name',outputCol='dept_name_Index')
SI_left = StringIndexer(inputCol='left',outputCol='left_Index')
SI_title = StringIndexer(inputCol='title',outputCol='title_Index')


# transform the data
df = SI_Last_performance_rating.fit(df).transform(df)
df = SI_dept_name.fit(df).transform(df)
df = SI_left.fit(df).transform(df)
df = SI_title.fit(df).transform(df)

In [29]:
df.select('Last_performance_rating','Last_performance_rating_Index','dept_name','dept_name_Index','left','left_Index','title','title_Index').show(10)

+-----------------------+-----------------------------+------------------+---------------+----+----------+---------------+-----------+
|Last_performance_rating|Last_performance_rating_Index|         dept_name|dept_name_Index|left|left_Index|          title|title_Index|
+-----------------------+-----------------------------+------------------+---------------+----+----------+---------------+-----------+
|                      A|                          1.0|           Finance|            8.0|   0|       0.0|          Staff|        0.0|
|                      C|                          2.0|        Production|            1.0|   0|       0.0|       Engineer|        2.0|
|                      A|                          1.0|        Production|            1.0|   0|       0.0|       Engineer|        2.0|
|                      C|                          2.0|   Human Resources|            7.0|   0|       0.0|          Staff|        0.0|
|                      B|                          0.0|

In [30]:
# create object and specify input and output column
OHE = OneHotEncoder(inputCols=['Last_performance_rating_Index', 'dept_name_Index','left_Index', 'title_Index'],outputCols=['Last_performance_rating_vec', 'dept_name_vec','left_vec', 'title_vec'])

# transform the data
df = OHE.fit(df).transform(df)

# view and transform the data
df.select('Last_performance_rating','Last_performance_rating_Index','Last_performance_rating_vec','dept_name','dept_name_Index','dept_name_vec','left','left_Index','left_vec','title','title_Index','title_vec').show(10)


+-----------------------+-----------------------------+---------------------------+------------------+---------------+-------------+----+----------+-------------+---------------+-----------+-------------+
|Last_performance_rating|Last_performance_rating_Index|Last_performance_rating_vec|         dept_name|dept_name_Index|dept_name_vec|left|left_Index|     left_vec|          title|title_Index|    title_vec|
+-----------------------+-----------------------------+---------------------------+------------------+---------------+-------------+----+----------+-------------+---------------+-----------+-------------+
|                      A|                          1.0|              (4,[1],[1.0])|           Finance|            8.0|    (8,[],[])|   0|       0.0|(1,[0],[1.0])|          Staff|        0.0|(6,[0],[1.0])|
|                      C|                          2.0|              (4,[2],[1.0])|        Production|            1.0|(8,[1],[1.0])|   0|       0.0|(1,[0],[1.0])|       Engineer|  

In [ ]:
df.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- emp_title_id: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- no_of_projects: integer (nullable = true)
 |-- Last_performance_rating: string (nullable = true)
 |-- left: integer (nullable = true)
 |-- last_date: string (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- title_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- Last_performance_rating_Index: double (nullable = false)
 |-- dept_name_Index: double (nullable = false)
 |-- left_Index: double (nullable = false)
 |-- title_Index: double (nullable = false)
 |-- Last_performance_rating_vec: vector (nullable = true)
 |-- dept_name_vec: vector (nullable = true)
 |-- left_vec: vector (nullable = 

In [31]:
featureCols = continuous_features + ['Last_performance_rating_vec', 'dept_name_vec','left_vec', 'title_vec']

In [32]:
featureCols

['no_of_projects',
 'salary',
 'Last_performance_rating_vec',
 'dept_name_vec',
 'left_vec',
 'title_vec']

In [33]:
assembler = VectorAssembler( inputCols = featureCols, outputCol = "features")

In [34]:
df_joins = assembler.transform( df )

In [36]:
df_joins.show( 10 , truncate=False)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+---------------+-----------------------------+---------------+----------+-----------+---------------------------+-------------+-------------+-------------+----------------------------------------------------+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex|hire_date |no_of_projects|Last_performance_rating|left|last_date|dept_no|dept_name         |salary|title_id|title          |Last_performance_rating_Index|dept_name_Index|left_Index|title_Index|Last_performance_rating_vec|dept_name_vec|left_vec     |title_vec    |features                                            |
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+---------------+-----------------------------+---------------+----------+-----------+--

In [38]:
df_joins = df_joins.withColumn( "label", df.left.cast( 'int' ) )

In [39]:
train_df, test_df = df_joins.randomSplit( [0.7, 0.3], seed = 42 )

In [40]:
train_df.show(5, truncate=False)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+------------------+-----------------------------+---------------+----------+-----------+---------------------------+-------------+-------------+-------------+---------------------------------------------------+-----+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex|hire_date |no_of_projects|Last_performance_rating|left|last_date|dept_no|dept_name         |salary|title_id|title             |Last_performance_rating_Index|dept_name_Index|left_Index|title_Index|Last_performance_rating_vec|dept_name_vec|left_vec     |title_vec    |features                                           |label|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+------------------+-----------------------------+---------------+------

In [41]:
from pyspark.ml.regression import LinearRegression

In [42]:
linreg = LinearRegression()

In [43]:
lm = linreg.fit( train_df )

In [44]:
lm.intercept

1.0000000000022824

In [45]:
lm.coefficients

DenseVector([-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0])

In [46]:
#Make predictions on train data and evaluate
y_pred_train = lm.transform(train_df)

In [47]:
y_pred_train.show(10)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+------------------+-----------------------------+---------------+----------+-----------+---------------------------+-------------+-------------+-------------+--------------------+-----+--------------------+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex| hire_date|no_of_projects|Last_performance_rating|left|last_date|dept_no|         dept_name|salary|title_id|             title|Last_performance_rating_Index|dept_name_Index|left_Index|title_Index|Last_performance_rating_vec|dept_name_vec|     left_vec|    title_vec|            features|label|          prediction|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+------------------+-----------------------------+---------------+----------+-----------+---

In [48]:
#Make predictions on test data and evaluate
y_pred_test = lm.transform( test_df )

In [49]:
y_pred_test.show(10, truncate=False)

+------+------------+----------+----------+-----------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+----------------+-----------------------------+---------------+----------+-----------+---------------------------+-------------+-------------+-------------+---------------------------------------------------+-----+-----------------------+
|emp_no|emp_title_id|birth_date|first_name|last_name  |sex|hire_date |no_of_projects|Last_performance_rating|left|last_date|dept_no|dept_name         |salary|title_id|title           |Last_performance_rating_Index|dept_name_Index|left_Index|title_Index|Last_performance_rating_vec|dept_name_vec|left_vec     |title_vec    |features                                           |label|prediction             |
+------+------------+----------+----------+-----------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+----------------+----

In [50]:
y_pred_test.select( 'features',  'label', 'prediction', 'left' ).show( 10 )

+--------------------+-----+--------------------+----+
|            features|label|          prediction|left|
+--------------------+-----+--------------------+----+
|(21,[0,1,3,13,14,...|    0|-1.11022302462515...|   0|
|(21,[0,1,4,13,14,...|    0|-6.66133814775093...|   0|
|(21,[0,1,2,10,14,...|    0|2.220446049250313...|   0|
|(21,[0,1,4,8,14,1...|    0|-6.66133814775093...|   0|
|(21,[0,1,2,10,14,...|    0|4.440892098500626...|   0|
|(21,[0,1,2,6,17],...|    1|   1.000000000000001|   1|
|(21,[0,1,3,6,14,1...|    0|-1.11022302462515...|   0|
|(21,[0,1,3,12,14,...|    0|-1.11022302462515...|   0|
|(21,[0,1,2,7,14,1...|    0|-4.44089209850062...|   0|
|(21,[0,1,3,6,14,1...|    0|-4.44089209850062...|   0|
+--------------------+-----+--------------------+----+
only showing top 10 rows



In [51]:
from pyspark.ml.evaluation import RegressionEvaluator

In [52]:
rmse_evaluator = RegressionEvaluator(labelCol="label",
                            predictionCol="prediction",
                            metricName="rmse" )

In [53]:
lm_rmse = rmse_evaluator.evaluate( y_pred_test)
lm_rmse

2.6024160854175806e-14

In [54]:
r2_evaluator = RegressionEvaluator(labelCol="label",
                            predictionCol="prediction",
                            metricName="r2" )
lm_r2 = r2_evaluator.evaluate( y_pred_test )
lm_r2

1.0

In [55]:
continuous_features

['no_of_projects', 'salary']

In [56]:
categorical_features

['last_performance_ratings', 'dept_name', 'left_', 'title']

In [57]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression

## Create indexers for the categorical features
indexers = [StringIndexer(inputCol=c, outputCol="{}_idx".format(c)) for c in categorical_features]

## encode the categorical features
encoders = [ OneHotEncoder(
      inputCol=idx.getOutputCol(),
      outputCol="{0}_enc".format(idx.getOutputCol())) for idx in indexers]

## Create vectors for all features categorical and continuous

assembler = VectorAssembler(
  inputCols=[enc.getOutputCol() for enc in encoders] + continuous_features,
  outputCol="features1")

## Initialize the linear model
lrModel = LinearRegression( maxIter = 10 )


## Create the pipeline with sequence of activities
#pipeline = Pipeline( stages=indexers + encoders + [assembler, lrModel ])
stages = indexers + encoders + [assembler, lrModel]

pipeline = Pipeline(stages=stages)

In [58]:
df_joins = df_joins.withColumn( "label", df.Last_performance_rating_Index.cast( 'int' ) )

In [59]:
training, testing = df_joins.randomSplit( [0.7, 0.3], seed = 42 )

In [60]:
training.show(5)

+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+------------------+-----------------------------+---------------+----------+-----------+---------------------------+-------------+-------------+-------------+--------------------+-----+
|emp_no|emp_title_id|birth_date|first_name|last_name|sex| hire_date|no_of_projects|Last_performance_rating|left|last_date|dept_no|         dept_name|salary|title_id|             title|Last_performance_rating_Index|dept_name_Index|left_Index|title_Index|Last_performance_rating_vec|dept_name_vec|     left_vec|    title_vec|            features|label|
+------+------------+----------+----------+---------+---+----------+--------------+-----------------------+----+---------+-------+------------------+------+--------+------------------+-----------------------------+---------------+----------+-----------+---------------------------+-------------+---

In [61]:
model = pipeline.fit( training)

In [ ]:
y_pred_train = model.transform(training)

In [ ]:
model.save("linear_regression.model")

In [ ]:
!zip -r /content/linear_regression_model.zip /content/linear_regression.model

  adding: content/linear_regression.model/ (stored 0%)
  adding: content/linear_regression.model/metadata/ (stored 0%)
  adding: content/linear_regression.model/metadata/.part-00000.crc (stored 0%)
  adding: content/linear_regression.model/metadata/._SUCCESS.crc (stored 0%)
  adding: content/linear_regression.model/metadata/_SUCCESS (stored 0%)
  adding: content/linear_regression.model/metadata/part-00000 (deflated 40%)
  adding: content/linear_regression.model/stages/ (stored 0%)
  adding: content/linear_regression.model/stages/08_VectorAssembler_754dc6ee495c/ (stored 0%)
  adding: content/linear_regression.model/stages/08_VectorAssembler_754dc6ee495c/metadata/ (stored 0%)
  adding: content/linear_regression.model/stages/08_VectorAssembler_754dc6ee495c/metadata/.part-00000.crc (stored 0%)
  adding: content/linear_regression.model/stages/08_VectorAssembler_754dc6ee495c/metadata/._SUCCESS.crc (stored 0%)
  adding: content/linear_regression.model/stages/08_VectorAssembler_754dc6ee495c/me